# Suggested Solution to Midterm (Question 3)


In [1]:
using Pkg
Pkg.activate(joinpath(pwd(),"..")) ;

  Activating project at `c:\Users\steve\Documents\GitHub\ECON627_UBC.jl`


## Load Main Packages

In [2]:
using Distributions, Random, Parameters, LinearAlgebra, PrettyTables

## Set Constants

In [3]:
β= [1.0 ; 1.0]
const ρ=0.9
const Σ=[1.0 ρ; ρ 1;]
const n = 100
const R = 10^4
const α = 0.05 ;

## Random Seed

In [4]:
Random.seed!(1234);

## DGP

The data generating process follows the structure of question 2. Where errors across equations have a correlation parameter of 0.9. Furthermore, errors are homoskedastic. 

In [5]:
function generate_data(n)
    #Define the Multivariate Normal Distribution instance
    mvnormal = MvNormal([0.0; 0.0], Σ)
    

    #DGP : Correlation across errors
     
    Errors=rand(mvnormal,n)'
    U1 = Errors[:,1]
    U2 = Errors[:,2]  
    U = vcat(U1, U2)
    
    X1 = rand(Normal(),n,1) 
    X2 = rand(Normal(),n,1)
    
    Z = hcat(X1, X2)
    
    X = vcat( hcat(X1,zeros(n,1)) , hcat(zeros(n,1), X2) ) 
    Y = X*β + U 
    
    return (Y = Y , X = X, Z =Z )
end

generate_data (generic function with 1 method)

In [6]:
function GMM(W, Y, X, Z)
    
    βGMM=(X'*Z*W*Z'*X)\(X'*Z*W*Z'*Y)
    Q = Z'*X/n
    
    se = sqrt(inv(Q'*W*Q)/n)
    
    return (βGMM = βGMM , se = se)
end

GMM (generic function with 1 method)

## Simulation

Both the equation-by-equation estimator and the system efficient estimator are consistent. To see that the system efficient estimator has lower variance, we can compute the average confidence interval length.

In [7]:
length_eqbyeq = zeros(1,2)
length_systemeff = zeros(1,2)

for r=1:R
    Y, X, Z = generate_data(n)
    # Equation by Equation 
    W1 = inv(kron([1 0 ; 0 1], (Z'Z)/n))
    Znew = kron([1 0 ; 0 1], Z)
        
    β_eqbyeq, se_eqbyeq = GMM(W1,Y,X,Znew)   
    
    # System Efficient 
    Uhat = Y - X*β_eqbyeq 
    Unew = vcat( hcat(Uhat[1:n] , zeros(n,1)) , hcat(zeros(n,1), Uhat[n+1:end] ) )
    Σhat = (Unew'Unew)/n 
    W2 = inv(kron( Σhat , (Z'Z)/n))
    
    β_systemeff, se_systemeff = GMM(W2,Y,X,Znew)   
   
    
    # Compute Confidence Intervals
    upper_eqbyeq = β_eqbyeq .- diag(se_eqbyeq)*quantile.(Normal(), α./2)/sqrt(n)
    lower_eqbyeq = β_eqbyeq .+ diag(se_eqbyeq)*quantile.(Normal(), α./2)/sqrt(n)
    
    upper_systemeff = β_systemeff .- diag(se_systemeff)*quantile.(Normal(), α./2)/sqrt(n)
    lower_systemeff = β_systemeff .+ diag(se_systemeff)*quantile.(Normal(), α./2)/sqrt(n)    
    
    length_eqbyeq .+= 1*(upper_eqbyeq .- lower_eqbyeq)'
    length_systemeff .+= 1*(upper_systemeff .- lower_systemeff)'
    
end

In [10]:
header = ["Parameter", "Equation-by-Equation",  "System Efficient"]
table = [ "β1"  length_eqbyeq[1]/R  length_systemeff[1]/R ;
"β2"  length_eqbyeq[2]/R  length_systemeff[2]/R ;
]
pretty_table(table; header)


┌───────────┬──────────────────────┬──────────────────┐
│ Parameter │ Equation-by-Equation │ System Efficient │
├───────────┼──────────────────────┼──────────────────┤
│        β1 │             0.039486 │        0.0392128 │
│        β2 │            0.0394937 │        0.0391938 │
└───────────┴──────────────────────┴──────────────────┘


As expected, the system efficient GMM yields the smallest standard errors (shortest confidence interval).